In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

from ucsbdistrict.inputs import CUP_inputs, weather_df, heatPump_df,districtModule_inputs,TES_inputs,dateTime_df,coldTank_schedule2_inputs_df,hotTank_schedule1_inputs_df,\
districtModule_inputs,groundReturn_schedule6_inputs_df, month_to_match,ocean_df, days_to_match,chilledCTOcean_schedule5_inputs_df


inputs 293738.9131106457


In [2]:
#temporarily importing ocean_temp_WS

ocean_df_WS = pd.read_excel('Ocean temp_WS.xlsx')
ocean_df_WS = pd.DataFrame(ocean_df_WS)
print(ocean_df_WS.columns)
ocean_df_WS_temporary = ocean_df_WS["Ocean Water Temp (°F)\nFor Water-Source"]

Index(['Ocean Water Temp (°F)\nFor Water-Source'], dtype='object')


In [3]:
ocean_df.head()

,\n\n\nYear,Month,Day,Surface Temp (°C),Surface Temp (°F),CombinedDate
0,2021,1,1,NaN,55.94,11
1,2021,1,2,13.3,55.94,21
2,2021,1,3,13.3,55.94,31
3,2021,1,4,13.3,55.94,41
4,2021,1,5,13.4,56.12,51


In [4]:
# Load the DataFrame from the pickle file
buildingMixing_output = pd.read_pickle('buildingMixing_output.pkl')

# Now 'df' contains the DataFrame loaded from the pickle file
buildingMixing_output.head()  # Verify the DataFrame

,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),Total Heating Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Cooling Load (Btu/h)
0,1.231706e+07,3.827353e+06,1.614441e+07,520.917695,103.0155,0.024464,54.000009,122.322571
1,9.027328e+06,3.236478e+06,1.226381e+07,389.913673,102.094763,0.174641,54.000061,873.211983
2,1.029800e+07,3.468767e+06,1.376677e+07,440.544528,102.501126,0.174641,54.000061,873.211983
3,1.133739e+07,3.165227e+06,1.450262e+07,472.558806,103.620896,0.174641,54.000061,873.211983
4,1.177647e+07,4.181198e+06,1.595766e+07,508.396115,102.223504,0.174641,54.000061,873.211983


In [5]:
# ##Heat pump input

# districtHWRT = buildingMixing_output["District HWRT (°F)"]
# districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
# districtHWSflow = buildingMixing_output["District HWS Flow (gpm)"]
# districtCHWSflow = buildingMixing_output["District CHWS Flow (gpm)"]
# HP_leavingHW = heatPump_df["Leaving Hot Water (°F)"]
# HP_enteringHW = heatPump_df["Entering Hot Water (°F)"]
# HP_enteringCHW = heatPump_df["Entering Cold Water (°F)"]
# HP_leavingCHW = heatPump_df["Leaving Cold Water (°F)"]
# HP_maxHeating = heatPump_df["Max Heating Output (Btu/h)"]
# HP_maxCooling = heatPump_df["Max Cooling Output (Btu/h)"]
# HW_districtSTP =  districtModule_inputs["Loop HW STP (°F)"]
# HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]
# HP_coolingCapacity=CUP_inputs["Heat Pumps"]["Cooling Capacity"]
# HP_waterSource=CUP_inputs["Heat Pumps"]["Water-Source Only %"]
# HP_airWaterSource=CUP_inputs["Heat Pumps"]["Air-Source + Water Source %"]





In [6]:
# from ucsbdistrict.heatPump import heatPump

# heatPump_instance = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,\
#                              HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
#                               HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling )

In [7]:
# heatPump_output_df = heatPump_instance.compute()

In [8]:
# print(HP_enteringHW - districtHWRT)

In [9]:
#TES inputs

HW_upperTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"]
HW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Volume"]
HW_TankVol = TES_inputs["Tank Properties"]["TES Hot Tank Volume"]
HW_wallLoss = TES_inputs["Tank Properties"]["TES Hot Wall-Losses"]
HW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Hot Thermocline/other-Losses"]
CHW_upperTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Volume"]
CHW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"]
CHW_TankVol = TES_inputs["Tank Properties"]["TES Cold Tank Volume"]
CHW_wallLoss = TES_inputs["Tank Properties"]["TES Cold Wall-Losses"]
CHW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Cold Thermocline/other-Losses"]

#parameters for second instance
thermoclineDischarge = TES_inputs["Tank Charging Characteristics"]["Fraction of Thermocline to discharge"]
stopCharging = TES_inputs["Tank Charging Characteristics"]["Stop Charging tank at ____ Capacity"]
totalHeating_load=buildingMixing_output["Total Heating Load (Btu/h)"]
#HP_heatingCapacity taken from heat pump inputs
districtHWRT = buildingMixing_output["District HWRT (°F)"]
hotMaxflow = TES_inputs["Tank Charging Characteristics"]["TES Hot Max Flow"]
coldMaxflow = TES_inputs["Tank Charging Characteristics"]["TES Cold Max Flow"]
HW_districtSTP =districtModule_inputs["Loop HW STP (°F)"]
HW_upperTankTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Temp"]
HW_lowerTankTemp = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Temp"]
HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]

chiller_maxCapacity=CUP_inputs["Electric Chillers"]["Max Capacity"]

##cold tank inputs
totalCooling_load=buildingMixing_output["Total Cooling Load (Btu/h)"]
districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
CHW_lowerTankTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Temp"]
CHW_upperTankTemp = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Temp"]
# HP_CHWST = heatPump_output_df["HP CHWST (°F)"]
conversion_galToLbs = TES_inputs["Conversion Rates"]["GAL to Lbs"]



In [10]:
#TES instance

# from ucsbdistrict.TES import TES

# TES_instance_one = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
#           CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
#             HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,totalHeating_load=totalHeating_load,HW_districtSTP=HW_districtSTP,\
#               districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp)

In [11]:
# TES_instance_one.compute()

In [12]:
#print TES inputs for checking
print(TES_inputs)

{'Conversion Rates': {'CUF to GAL': 7.48, 'GAL to Lbs': 8.34}, 'Initial Conditions': {'Starting TES Hot Charge': 0.95, 'TES Hot Tank Upper Tank Temp': 170.0, 'TES Hot Tank Lower Tank Temp': 80.0, 'TES Hot Tank Upper Tank Volume': 251146.77070960205, 'TES Hot Thermocline thickness': 2.0, 'TES Hot Thermocline volume': 29373.891311064566, 'TES Hot Tank Lower Tank Volume': 13218.25108997908, 'Starting TES Cold Charge': 0.95, 'TES Cold Tank Upper Tank Temp': 60.0, 'TES Cold Tank Lower Tank Temp': 44.0, 'TES Cold Tank Upper Tank Volume': 61332.68505750294, 'TES Cold Thermocline thickness': 2.0, 'TES Cold Thermocline volume': 42298.40348793298, 'TES Cold Tank Lower Tank Volume': 1165321.0160925535}, 'Tank Charging Characteristics': {'TES Hot Capacity': 220480.4281808507, 'TES Cold Capacity': 169328.96884289332, 'TES Hot Max % of Total Tank Volume moved in 1 hr': 0.08, 'TES Cold Max % of Total Tank Volume moved in 1 hr': 0.08, 'TES Hot Max Flow': 395, 'TES Cold Max Flow': 1695, 'Fraction of Th

In [13]:
#CUP inputs
timeStamp = dateTime_df
hotTank_schedule = hotTank_schedule1_inputs_df 
coldTank_schedule = coldTank_schedule2_inputs_df

totalCooling_load=buildingMixing_output["Total Cooling Load (Btu/h)"]
hotTank_schedule.head()

,Schedules,Load Shift Control,Month,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,Total
0,Schedules,Load Shift Control - Hot Tank by month,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
1,Schedules,Load Shift Control - Hot Tank by month,2,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
2,Schedules,Load Shift Control - Hot Tank by month,3,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
3,Schedules,Load Shift Control - Hot Tank by month,4,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
4,Schedules,Load Shift Control - Hot Tank by month,5,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13


In [14]:
type(dateTime_df)
# print(timeStamp)
timeStamp.head()

,Date and Time
0,2010-01-01 00:00:00
1,2010-01-01 01:00:00
2,2010-01-01 02:00:00
3,2010-01-01 03:00:00
4,2010-01-01 04:00:00


In [15]:

from ucsbdistrict.CUP import CUP
CUP_instance = CUP(timeStamp=timeStamp,hotTank_schedule=hotTank_schedule,coldTank_schedule=coldTank_schedule,totalHeating_load=totalHeating_load,\
                   totalCooling_load=totalCooling_load)

In [16]:
CUP_output_df = CUP_instance.compute_CUP()
CUP_output_df.head()

,Hot Load Shift Charging,Cold Load Shift Charging,Hot Load Shift Hours,Cold Load Shift Hours,Shift Count,Predicted Day's Heating Load (Btu/h),Predicted Day's Cooling Load (Btu/h),Predicted Heating Load in Load Shift Window (Btu/h),Predicted Cooling Load in Load Shift Window (Btu/h)
0,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
1,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
2,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
3,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004
4,False,True,13,9,0,4.658689e+08,6.708789e+07,2.770624e+08,5728.561004


In [17]:
# CUP_output_df.to_pickle('CUP_output_df.pkl')

In [18]:
# HotLoadShiftCharging = CUP_instance["Hot Load Shift Charging"]

# print(HotLoadShiftCharging)

In [19]:
from ucsbdistrict.TES import TES
TES_instance_two = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
                      CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
                      HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,CUP_output_df=CUP_output_df,totalHeating_load=totalHeating_load,\
                        HW_districtSTP=HW_districtSTP,districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp,HW_lowerTankTemp=HW_lowerTankTemp,totalCooling_load=totalCooling_load,\
                      districtCHWRT = districtCHWRT, CHW_lowerTankTemp = CHW_lowerTankTemp, CHW_upperTankTemp=CHW_upperTankTemp,conversion_galToLbs=conversion_galToLbs)
# TES_instance_two.compute_2()


In [20]:
TES_H_ouput1_df = TES_instance_two.TES_H_calculate()
TES_H_ouput1_df.head(15)

TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
dtype: float64 0    482.034245
1    361.203540
dtype: float64 0    13214.285614652086
1     36903.21132896769
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
dtype: float64 0    482.034245
1    361.203540
2    407.911055
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
3    98.964994
dtype: float64 0    482.034245
1    361.203540
2    407.911055
3    436.963356
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
3      81965.8353813555
dtype: object
TES_H_temp_tc 0     80.000000
1     94.776593
2     97.484229
3     98.964994
4    100.093330
dtype: float64 0    482.034245
1 

,TES Hot Capacity hr-1 (%),TES Hot Th after losses (Gal)_supply temp,TES Hot Tc after losses (Gal)_return temp,TES Hot Thermocline (Gal),TES Hot Status,Hot Load-Shift Charge Rate (gpm),Flow into TES Hot Th (gpm),TES Hot Th Previous Hour (°F),TES Flow to meet Campus Heating (gpm),Flow out of TES Hot Th (gpm),TES Hot Th (°F),TES Hot Th (Gal),TES Hot Tc (Gal),TES Hot Capacity (%),districtHWRT,TES Hot Tc Previous Hour(°F),TES Hot Tc (°F)
0,0.855000,251071.42667838917,13214.285614652086,29453.200818,Discharge,0.0,0,170.0,482.034245,395.0,170.0,227371.426678,36914.285615,0.774060,103.0155,80.000000,94.776593
1,0.774060,227303.21525038563,36903.21132896769,29532.486531,Discharge,0.0,0,170.0,361.203540,361.20354019660437,170.0,205631.002839,58575.423741,0.700047,102.094763,94.776593,97.484229
2,0.700047,205178.61463234448,58446.55780853428,30113.740670,Discharge,0.0,0,170.0,407.911055,395.0,170.0,181478.614632,82146.557809,0.617823,102.501126,97.484229,98.964994
3,0.617823,181079.36168015332,81965.8353813555,30693.716049,Discharge,0.0,0,170.0,436.963356,395.0,170.0,157379.361680,105665.835381,0.535780,103.620896,98.964994,100.093330
4,0.535780,157033.127084457,105433.37054351652,31272.415483,Discharge,0.0,0,170.0,470.890774,395.0,170.0,133333.127084,129133.370544,0.453917,102.223504,100.093330,100.543522
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.331745,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.920655,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.0415,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.192218,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94.654645,NaN,NaN


In [21]:

CT_CUP_value = chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]== "Cooling Tower Approach","CUP"]
CT_CUP_month = chilledCTOcean_schedule5_inputs_df.loc[chilledCTOcean_schedule5_inputs_df["Schedules Category"]== "Cooling Tower Approach","Month"]

In [22]:
##Heat pump input

districtHWRT = buildingMixing_output["District HWRT (°F)"]
districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
districtHWSflow = buildingMixing_output["District HWS Flow (gpm)"]
districtCHWSflow = buildingMixing_output["District CHWS Flow (gpm)"]
HP_leavingHW = heatPump_df["Leaving Hot Water (°F)"]
HP_enteringHW = heatPump_df["Entering Hot Water (°F)"]
HP_enteringCHW = heatPump_df["Entering Cold Water (°F)"]
HP_leavingCHW = heatPump_df["Leaving Cold Water (°F)"]
HP_maxHeating = heatPump_df["Max Heating Output (Btu/h)"]
HP_maxCooling = heatPump_df["Max Cooling Output (Btu/h)"]
HW_districtSTP =  districtModule_inputs["Loop HW STP (°F)"]
HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]
HP_coolingCapacity=CUP_inputs["Heat Pumps"]["Cooling Capacity"]
HP_waterSource=CUP_inputs["Heat Pumps"]["Water-Source Only %"]
HP_airWaterSource=CUP_inputs["Heat Pumps"]["Air-Source + Water Source %"]
dryBulb_temp =  weather_df["Dry Bulb Temp (°F)"]
wetBulb_temp = weather_df["Wet Bulb Temp (°F)"]

grReturn_D_LWT = groundReturn_schedule6_inputs_df["Ht Ex - LWT*"]
grReturn_HtRej_LWT = groundReturn_schedule6_inputs_df["Ht Rej - LWT*"]
gr_months = groundReturn_schedule6_inputs_df["Schedules Month"]

month_to_match = month_to_match

HP_grSource_capacity=CUP_inputs["Heat Source/Sink"]["Ground-source Capacity"]
HP_ocean_capacity=CUP_inputs["Heat Source/Sink"]["Ocean Capacity"]
HP_CT_capacity=CUP_inputs["Heat Source/Sink"]["Cooling Tower Capacity"]


In [23]:
#Heat pump inputs from TES

TES_H_shiftChargeRate = TES_H_ouput1_df["Hot Load-Shift Charge Rate (gpm)"]
TES_H_flowOut = TES_H_ouput1_df["Flow out of TES Hot Th (gpm)"]



In [24]:
from ucsbdistrict.heatPump import heatPump

heatPump_instance = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,districtHWSflow=districtHWSflow,\
                            HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
                            HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling,TES_H_shiftChargeRate=TES_H_shiftChargeRate,\
                            TES_H_flowOut=TES_H_flowOut,HP_heatingCapacity=HP_heatingCapacity,HP_coolingCapacity=HP_coolingCapacity,districtCHWSflow=districtCHWSflow,\
                            grReturn_D_LWT=grReturn_D_LWT,gr_months=gr_months,month_to_match=month_to_match,dryBulb_temp=dryBulb_temp,ocean_df=ocean_df,\
                            days_to_match=days_to_match,HP_waterSource=HP_waterSource,HP_airWaterSource=HP_airWaterSource,ocean_df_WS_temporary=ocean_df_WS_temporary,\
                            HP_grSource_capacity=HP_grSource_capacity,HP_ocean_capacity=HP_ocean_capacity,chiller_maxCapacity=chiller_maxCapacity,\
                            grReturn_HtRej_LWT=grReturn_HtRej_LWT,wetBulb_temp=wetBulb_temp,CT_CUP_value=CT_CUP_value,CT_CUP_month=CT_CUP_month,HP_CT_capacity=HP_CT_capacity)

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pydantic\_internal\_model_construction.py:53: UserWarning: `reqCoolingOutput_gpm` overrides an existing Pydantic `@computed_field` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


In [25]:
heatPump_output_df = heatPump_instance.compute()
heatPump_output_df.head(15)

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


,HP CHWST (°F),Max Unitary Heating Output (Btu/h),Max Unitary Cooling Output (Btu/h),Heat:Cool,Required Heating Output (Btu/h),HWSFLow,self.HW_districtSTP,self.districtHWRT,Possible Recovered CHW (Btu/h),Possible Recovered CHW (gpm)
0,44,145.6,96.1,1.515088,4217266.902527,520.917695,170,103.0155,2783512.014648,556.701926
1,44,145.6,96.1,1.515088,974784.181853,389.913673,170,102.094763,643384.339808,128.67608
2,44,145.6,96.1,1.515088,1537102.168396,440.544528,170,102.501126,1014529.659223,202.90469
3,44,145.6,96.1,1.515088,2574142.022344,472.558806,170,103.620896,1699004.452935,339.798811
4,44,145.6,96.1,1.515088,3842795.657386,508.396115,170,102.223504,2536350.705184,507.267036
5,44,145.6,96.1,1.515088,70000000.0,479.779617,170,99.331745,46201923.076923,9240.320138
6,44,145.6,96.1,1.515088,70000000.0,509.883985,170,95.920655,46201923.076923,9240.312215
7,44,145.6,96.1,1.515088,70000000.0,521.839294,170,93.0415,46201923.076923,9240.304293
8,44,145.6,96.1,1.515088,70000000.0,520.53427,170,91.192218,46201923.076923,9240.360848
9,44,145.6,96.1,1.515088,70000000.0,685.206776,170,94.654645,46201923.076923,9068.773702


In [26]:
HP_possRecCHW=heatPump_output_df["Possible Recovered CHW (gpm)"]
HP_CHWST=heatPump_output_df["HP CHWST (°F)"]
TES_hotCapacityHr =TES_H_ouput1_df["TES Hot Capacity hr-1 (%)"]

In [27]:
TES_instance_three = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
                      CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
                      HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,CUP_output_df=CUP_output_df,totalHeating_load=totalHeating_load,\
                        HW_districtSTP=HW_districtSTP,districtHWRT=districtHWRT,hotMaxflow=hotMaxflow,coldMaxflow=coldMaxflow,HW_upperTankTemp=HW_upperTankTemp,HW_lowerTankTemp=HW_lowerTankTemp,totalCooling_load=totalCooling_load,\
                      districtCHWRT = districtCHWRT, CHW_lowerTankTemp = CHW_lowerTankTemp, CHW_upperTankTemp=CHW_upperTankTemp,HP_possRecCHW=HP_possRecCHW,HP_CHWST=HP_CHWST, TES_hotCapacityHr=TES_hotCapacityHr,\
                      conversion_galToLbs=conversion_galToLbs)
# TES_instance_two.compute_2()


In [28]:
TES_C_ouput1_df = TES_instance_three.TES_C_calculate()
TES_C_ouput1_df.head(20)

,TES Cold Capacity hr-1 (%),TES Cold Th after losses (Gal)_supply temp,TES Cold Tc after losses (Gal)_return temp,TES Cold Thermocline (Gal),TES Cold Charge Window Adjustment,TES Cold Status,Cold Load-Shift Charge Rate (gpm),TES Cold Tc Previous Hour (°F),TES Flow to meet Campus Heating (gpm),HP Flow to meet Campus Cooling (gpm),Flow into TES Cold Tc (gpm),Flow out of TES Cold Tc (gpm),TES Cold Th (Gal),TES Cold Tc (Gal),TES Cold Tc (°F),TES Cold Capacity (%),TES Cold Th (°F),TES Cold Th Previous Hour (°F)
0,0.918333,61320.41852049144,1165087.951889335,42543.734228,False,Charge,556.677461132728,44.0,0.024464,0.024464,556.677461132728,0.0,27919.770853,1.198489e+06,44.0,0.944471,60.0,60.0
1,0.944471,27914.18689835725,1198248.9018373874,42789.015902,False,Charge,128.50143907397953,44.0,0.174641,0.174641,128.50143907397953,0.0,20204.100554,1.205959e+06,44.0,0.950358,60.0,60.0
2,0.950358,20196.01891369691,1205476.6045865535,43279.481138,False,Charge,202.73004864186316,44.0,0.174641,0.174641,202.73004864186316,0.0,8032.215995,1.217640e+06,44.0,0.959564,60.0,60.0
3,0.959564,8029.003108787045,1217153.3513420632,43769.750187,False,Charge,339.62416952573454,44.0,0.174641,0.174641,339.62416952573454,0.0,0.000000,1.237531e+06,44.0,0.975238,60.0,60.0
4,0.975238,0.0,1237035.7891930018,31916.315445,False,Hold,0.0,44.0,0.174641,0.174641,0.0,0.0,0.000000,1.237036e+06,44.0,0.974848,60.0,60.0
5,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
filtered_df = TES_C_ouput1_df.loc[4320:4344,["TES Cold Capacity hr-1 (%)","Flow into TES Cold Tc (gpm)","Flow out of TES Cold Tc (gpm)"]]
# Sum the values of the filtered columns
sum_values = filtered_df.sum()
filtered_df.head(24)
print(sum_values)

TES Cold Capacity hr-1 (%)       0.0
Flow into TES Cold Tc (gpm)        0
Flow out of TES Cold Tc (gpm)      0
dtype: object


In [30]:
TES_HC_ouput1_df = TES_instance_three.TES_HC_calculate()
TES_HC_ouput1_df.head(20)

TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
dtype: float64 0    482.034245
1    361.203540
dtype: float64 0    13214.285614652086
1     36903.21132896769
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
dtype: float64 0    482.034245
1    361.203540
2    407.911055
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
3    98.964994
dtype: float64 0    482.034245
1    361.203540
2    407.911055
3    436.963356
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
3      81965.8353813555
dtype: object
TES_H_temp_tc 0     80.000000
1     94.776593
2     97.484229
3     98.964994
4    100.093330
dtype: float64 0    482.034245
1 

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.0
dtype: float64 0    395.0
dtype: object 0    13214.285614652086
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
dtype: float64 0    482.034245
1    361.203540
dtype: float64 0    13214.285614652086
1     36903.21132896769
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
dtype: float64 0    482.034245
1    361.203540
2    407.911055
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
dtype: object
TES_H_temp_tc 0    80.000000
1    94.776593
2    97.484229
3    98.964994
dtype: float64 0    482.034245
1    361.203540
2    407.911055
3    436.963356
dtype: float64 0    13214.285614652086
1     36903.21132896769
2     58446.55780853428
3      81965.8353813555
dtype: object
TES_H_temp_tc 0     80.000000
1     94.776593
2     97.484229
3     98.964994
4    100.093330
dtype: float64 0    482.034245
1 

,TES Hot Binary Status,TES Cold Binary Status,Temp out of Hot Tank (°F),Energy out of Hot Tank (Btus),Temp out of Cold Tank (°F),Energy out of Cold Tank (Btus),TES Hot Thermocline (Gal),TES Hot Capacity (kBtu),TES Cold Capacity (kBtu),check
0,1,0,170.0,-13229438.656923,60.000000,2783389.692078,29453.200818,127021.212601,99954.034901,0.0
1,1,0,170.0,-12263806.057488,60.000000,642511.127826,29532.486531,116454.939867,100577.595194,0.0
2,1,0,170.0,-13331027.585421,60.000000,1013656.44724,30113.740670,102161.681644,101551.831529,0.0
3,1,0,170.0,-13109873.111014,55.152113,1698131.240952,30693.716049,87125.486949,103210.700542,0.0
4,1,0,170.0,-13385857.946774,0.000000,0.0,31272.415483,75367.346894,103169.416262,0.0
5,0,0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
6,0,0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
7,0,0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
8,0,0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
9,0,0,0.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [31]:
TES_C_shiftChargeRate = TES_C_ouput1_df["Cold Load-Shift Charge Rate (gpm)"]
TES_C_flowOut = TES_C_ouput1_df["Flow out of TES Cold Tc (gpm)"]
TES_H_tempOut= TES_HC_ouput1_df["Temp out of Hot Tank (°F)"]
TES_C_tempOut = TES_HC_ouput1_df["Temp out of Cold Tank (°F)"]

In [32]:
from ucsbdistrict.heatPump import heatPump

heatPump_instance_two = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,districtHWSflow=districtHWSflow,\
                            HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
                            HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling,TES_H_shiftChargeRate=TES_H_shiftChargeRate,\
                            TES_H_flowOut=TES_H_flowOut,HP_heatingCapacity=HP_heatingCapacity,HP_coolingCapacity=HP_coolingCapacity,TES_C_shiftChargeRate=TES_C_shiftChargeRate,\
                            TES_C_flowOut = TES_C_flowOut, TES_H_tempOut=TES_H_tempOut, TES_C_tempOut=TES_C_tempOut,districtCHWSflow=districtCHWSflow,\
                            grReturn_D_LWT=grReturn_D_LWT,gr_months=gr_months,month_to_match=month_to_match,dryBulb_temp=dryBulb_temp,ocean_df=ocean_df,\
                            days_to_match=days_to_match,HP_waterSource=HP_waterSource,HP_airWaterSource=HP_airWaterSource,ocean_df_WS_temporary=ocean_df_WS_temporary,\
                            HP_grSource_capacity=HP_grSource_capacity,HP_ocean_capacity=HP_ocean_capacity,chiller_maxCapacity=chiller_maxCapacity,\
                            grReturn_HtRej_LWT=grReturn_HtRej_LWT,wetBulb_temp=wetBulb_temp,CT_CUP_value=CT_CUP_value,CT_CUP_month=CT_CUP_month,HP_CT_capacity=HP_CT_capacity)

In [33]:
heatPump_output_df_2 = heatPump_instance_two.compute2()
heatPump_output_df_2.head(15)

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)


c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invali

self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
checkP 0       70
1       70
2       70
3       70
4       70
        ..
8755    75
8756    75
8757    75
8758    75
8759    75
Name: Ht Rej - LWT*, Length: 8760, dtype: int64 0       57.501190
1       57.524185
2       57.515631
3       56.960694
4       56.592221
          ...    
8755    65.984365
8756    65.984365
8757    65.984365
8758    65.984365
8759    57.515631
Name: cooler_CWRT, Length: 8760, dtype: float64 0       55.9399
1       55.9399
2       55.9399
3       55.9399
4       55.9399
         ...   
8755    55.2199
8756    55.2199
8757    55

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\ucsbdistrict\heatPump.py:513: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[25000000.0 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0
 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0
 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0
 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0 25000000.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result[condition1] = np.minimum(L3[condition1], GS_Cap)
c:\Users\nikita.khatwani\Documents\UCSB\Di

self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]


...

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\ucsbdistrict\heatPump.py:541: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.0 0.0 0.0 ... 25000000.0 25000000.0 25000000.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  result[condition1] = np.minimum(L3[condition1

self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
self.CT_CUP_month.values [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.]
HW_indices [ 0  0  0 ... 11 11 11]
checkP 0       70
1       70
2       70
3       70
4       70
        ..
8755    75
8756    75
8757    75
8758    75
8759    75
Name: Ht Rej - LWT*, Length: 8760, dtype: int64 0       57.501190
1       57.524185
2       57.515631
3       56.960694
4       56.592221
          ...    
8755    65.984365
8756    65.984365
8757    65.984365
8758    65.984365
8759    57.515631
Name: cooler_CWRT, Length: 8760, dtype: float64 0       55.9399
1       55.9399
2       55.9399
3       55.9399
4       55.9399
         ...   
8755    55.2199
8756    55.2199
8757    55.2199
8758    55.2199
8759    55.2199
Name: Ocean Water Temp (°F)\nFor Water-Source, Length: 8760, dtype: floa

c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\nikita.khatwani\Documents\UCSB\District sytem - Engineering model\District system - Engineering model\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invali

index_aq 2 11
index_aq 2 12
index_aq 2 13
index_aq 2 14
index_aq 2 19
index_aq 3 20
index_aq 3 21
index_aq 3 22
index_aq 3 23
index_aq 3 24
index_aq 3 25
index_aq 3 26
index_aq 3 27
index_aq 3 28
index_aq 3 29
index_aq 3 30
index_aq 3 31
index_aq 3 32
index_aq 3 46
index_aq 3 47
index_aq 3 48
index_aq 2 111
index_aq 3 116
index_aq 3 117
index_aq 3 118
index_aq 3 119
index_aq 3 120
index_aq 3 121
index_aq 3 122
index_aq 3 123
index_aq 3 124
index_aq 3 125
index_aq 3 126
index_aq 3 127
index_aq 3 128
index_aq 2 133
index_aq 2 134
index_aq 3 138
index_aq 3 139
index_aq 3 140
index_aq 3 141
index_aq 3 142
index_aq 3 143
index_aq 3 144
index_aq 3 145
index_aq 3 146
index_aq 3 147
index_aq 3 148
index_aq 3 149
index_aq 3 150
index_aq 3 151
index_aq 3 152
index_aq 2 155
index_aq 2 156
index_aq 2 157
index_aq 2 158
index_aq 2 159
index_aq 3 162
index_aq 3 163
index_aq 3 164
index_aq 3 165
index_aq 3 166
index_aq 3 167
index_aq 3 168
index_aq 3 169
index_aq 3 170
index_aq 3 171
index_aq 2 179
i

,Required Heating Output (gpm),Required Cooling Output (Btu/h),Required Cooling Output (gpm),"Heat Pumps Heating+Cooling Capacity_(Cooling, Btu/h)","Heat Pumps Heating+Cooling Capacity_(Heating, Btu/h)",Total Heat Pump Heating Only Capacity,Ground Water Return Temp (°F),Ambient Dry-bulb Temp (°F),Ocean Water Temp (°F),Heat Pumps Heating Only Capacity_Water-Source (Btu/h),...,Ground Source Max (Btu/h),Chiller_Heat Rejection-Ocean Source (Btu/h),Chiller_Heat Rejection-Tower Source (Btu/h),Ocean Source Max (Btu/h),Tower Source Max (Btu/h),Air Source Max (Btu/h),Heat Pumps Cooling Only Capacity_Air-Source (Btu/h),Heat Pumps Cooling Only Capacity_Ground-Source (Btu/h),Heat Pumps Cooling Only Capacity_Ocean-Source (Btu/h),Heat Pumps Cooling Only Capacity_Tower-Source (Btu/h)
0,125.917695,2783512.014648,556.701926,2783512.014648,4217266.902527,0.0,60,54.14,55.9399,0.0,...,35000000.0,0.0,0.0,50000000.0,0,25000000.0,0.000000e+00,0.0,0.0,0.0
1,28.710133,643384.339808,128.67608,643384.339808,974784.181853,0.0,60,55.76,55.9399,0.0,...,35000000.0,0.0,0.0,50000000.0,0,25000000.0,0.000000e+00,0.0,0.0,0.0
2,45.544528,1014529.659223,202.90469,1014529.659223,1537102.168396,0.0,60,53.96,55.9399,0.0,...,35000000.0,0.0,0.0,50000000.0,0,25000000.0,0.000000e+00,0.0,0.0,0.0
3,77.558806,1699004.452935,339.798811,1699004.452935,2574142.022344,0.0,60,53.96,55.9399,0.0,...,35000000.0,0.0,0.0,50000000.0,0,25000000.0,0.000000e+00,0.0,0.0,0.0
4,113.396115,873.211983,0.174641,873.211983,1322.993389,3841472.663997,60,53.96,55.9399,3841472.663997,...,35000000.0,0.0,0.0,50000000.0,0,25000000.0,0.000000e+00,0.0,0.0,0.0
5,1981.087559,NaN,NaN,46201923.076923,70000000.0,0.0,60,53.96,55.9399,0.0,...,35000000.0,25000000.0,0.0,25000000.0,0,25000000.0,3.798077e+06,0.0,0.0,0.0
6,1889.865524,NaN,NaN,46201923.076923,70000000.0,0.0,60,54.14,55.9399,0.0,...,35000000.0,25000000.0,0.0,25000000.0,0,25000000.0,3.798077e+06,0.0,0.0,0.0
7,1819.162271,NaN,NaN,46201923.076923,70000000.0,0.0,60,55.22,55.9399,0.0,...,35000000.0,25000000.0,0.0,25000000.0,0,25000000.0,3.798077e+06,0.0,0.0,0.0
8,1776.474313,NaN,NaN,46201923.076923,70000000.0,0.0,60,56.30,55.9399,0.0,...,35000000.0,25000000.0,0.0,25000000.0,0,0.0,0.000000e+00,0.0,0.0,0.0
9,1858.110562,NaN,NaN,46201923.076923,70000000.0,0.0,60,59.18,55.9399,0.0,...,35000000.0,25000000.0,0.0,25000000.0,0,0.0,0.000000e+00,0.0,0.0,0.0


In [34]:
filtered_df = heatPump_output_df_2.iloc[:25,list(range(14, 22)) + list(range(26, 29))]
filtered_df.head(150)

,cooler_CWRT,Ground Water Return Temp (°F) For Water-Source Heat Rejection,Priority 1_Heat Sink,Priority 2_Heat Sink,Priority 3_Heat Sink,Priority 4_Heat Sink,Chiller_Heat Rejection-Ground Source (Btu/h),Ground Source Max (Btu/h),Air Source Max (Btu/h),Heat Pumps Cooling Only Capacity_Air-Source (Btu/h),Heat Pumps Cooling Only Capacity_Ground-Source (Btu/h)
0,57.501190,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,0.000000e+00,0.0
1,57.524185,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,0.000000e+00,0.0
2,57.515631,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,0.000000e+00,0.0
3,56.960694,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,0.000000e+00,0.0
4,56.592221,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,0.000000e+00,0.0
5,56.960694,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,3.798077e+06,0.0
6,57.130517,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,3.798077e+06,0.0
7,58.149453,70,Air Source,Ocean Source,Tower Source,Ground Source,0,35000000.0,25000000.0,3.798077e+06,0.0
8,58.595413,70,Ocean Source,Air Source,Tower Source,Ground Source,0,35000000.0,0.0,0.000000e+00,0.0
9,59.891229,70,Ocean Source,Air Source,Tower Source,Ground Source,0,35000000.0,0.0,0.000000e+00,0.0
